# New design

ttk.Frame &rarr; ListBoxFrame, ARCFrame, EncDecFrame

In [1]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox

# Skeleton &rarr; actual code

## 1. ListBox + ARCFrame integration = ListBoxFrame

The listbox and the Add-Remove-Clear frame will be used as
one single frame.

### 1.1 ListBox

In [43]:
class ListBox(tk.Listbox):
    """The List box.
    
    add -- Add items
    remove -- remove items
    clear -- clear items
    get -- get all items
    """
    def __init__(self, *args, master, **kwargs):
        super().__init__(*args, master=master, **kwargs)
        self.__items = tk.StringVar(master, name="__items")
        self.config(listvariable=self.__items)
        
        # x and y scrollbars with correct orientation
        xscrollbar = ttk.Scrollbar(self, orient='horizontal')
        yscrollbar = ttk.Scrollbar(self, orient='vertical')

        # pack them on the listbox
        xscrollbar.pack(side='bottom', fill='x')
        yscrollbar.pack(side='right', fill='y')

        # configure the listbox
        self.configure(
            xscrollcommand=xscrollbar.set,
            yscrollcommand=yscrollbar.set,
        )

        # set the scrollbar commands
        xscrollbar.config(command=self.xview)
        yscrollbar.config(command=self.yview)

    @property
    def items(self):
        return self.getvar('__items')

    def get(self):
        """Get the items from the ListBox"""
        return self._items.get()

    def add(self, item):
        """Add a unique item in the ListBox"""
        if item not in self.items:
            self.insert('end', item)
    
    def add_many(self, *items):
        """Add several items at once."""
        for item in items:
            self.add(item)

    def clear(self):
        """Clear the ListBox"""
        self.delete(0, 'end')

    remove_all = clear

    def remove(self):
        """Remove a single selected item from the ListBox"""
        try:
            idx = self.curselection()[0]
            self.delete(idx)
        except IndexError:
            messagebox.showerror("Error", "No file selected.")

### 1.2 ARCFrame

In [102]:
class ARCFrame(ttk.Frame):
    """Add, Remove, Clear Frame;
        * controls ListBoxFrame
    
    listbox -- the listbox to control
    on_add -- Add ctrl.
    on_remove -- rm ctrl.
    on_clear -- clearing ctrl. Aliased to 'Remove All'
    """
    def __init__(self, *args, master, listbox=None, **kwargs):
        super().__init__(*args, master=master, **kwargs)
        if not hasattr(self, "_listbox"):
            if listbox is None:
                raise TypeError("listbox is required")
            self._listbox = listbox  # don't pack: EIBTI

        self._badd = ttk.Button(
            self,
            text="Add",
            command=self.on_add,
        )
        self._bremove = ttk.Button(
            self,
            text="Remove",
            command=self.on_remove,
        )
        self._bclear = ttk.Button(
            self,
            text="Clear",
            command=self.on_clear,
        )

        self._badd.pack(anchor=tk.E, side=tk.LEFT)
        self._bremove.pack(anchor=tk.N, side=tk.LEFT, padx=80)
        self._bclear.pack(anchor=tk.W, side=tk.RIGHT)

    def on_add(self):
        """Open a DialogBox to select item(s) and add it to the listbox."""
        filepath = filedialog.askopenfilenames()
        if not filepath:
            return
        self._listbox.add_many(*filepath)

    def on_remove(self):
        """Remove a selected item from the listbox (Button mapped)"""
        self._listbox.remove()

    def on_clear(self):
        """Clear (or remove_all) the listbox (button mapped)"""
        self._listbox.clear()

    on_remove_all = on_clear

### * ListBoxFrame: Not to be used, but good for testing out things.

~The ListBox and the ARC-buttons are merged together in one frame.~
~They can be modified here.~

### <font color=red>The listbox accepting frames must be subclassed.</font>

In [58]:
class ListBoxFrame(ARCFrame, ttk.Frame):
    def __init__(self, *args, master, **kwargs):
        """Integrate listbox and ARC-buttons. Customize as required."""
        self.listbox = ListBox(*args, master=master, **kwargs)
        self.listbox.pack()
        super().__init__(*args, master=master, listbox=self.listbox, **kwargs)

In [59]:
root = tk.Tk()
try:
    lf = ListBoxFrame(master=root)
    lf.pack()

    root.mainloop()
except tk.TclError:
    root.destroy()

## 1.3 EncDecFrame: This may change!

Control the encryption and decryption.

1. Fetch the file names from ListBox,
2. Set up a **Thread-pool** or **process-pool**, (in a separate design?)
3. Yield as operate,
4. Update with colors,
5. Show stat in the end


### Problem:

The EncDecFrame needs to control the listbox, but unfortunately, that will
be used by the ListBoxFrame. That means, there will be no ***global***
reference to the ListBox object.

### Possible solution 1:

Since ListBox can be shared by both ARCFrame and EncDecFrame, without affecting
each other, i.e.:

                     listbox  <-- shared
                        |
              |---------|---------|
              |                   |
           ARCFrame          EncDecFrame

Subclassing it as:~
              
           ARCFrame          EncDecFrame  <-- * Both take listbox value
              |                   |
              |---------|---------|
                        |
                   ControlFrame (or some better name :p)
                   
will provide a good control over both the frames.

### Possible solution 2:

Instead of subclassing, make `ControlFrame` a simple `ttk.Frame` and set up
the `ARCFrame` and `EncDecFrame` there.

> There is more than one way to do it.
>> And subclassing is not necessarily the one among them.

### But following solution 1 causes Problem 2:

The superclasses require listbox value, but `super()` only goes 1 class up.
After that, the listbox is lost, and `TypeError` is raised.

### Possible Solution:

Make `listbox` an assumed variable. The superclasses must expect that the listbox
is coming if the subclass has a `self._listbox` variable.

> This scheme is really irritating and I want to get rid of this.

### Then, what is the best solution?

#### [Solution 2 of Problem 1](#Possible-solution-2:)

In [60]:
class EncDecFrame(ttk.Frame):
    """Encrypt / Decrypt buttons w/ a password entry.
        * Controls ListBox

    on_encrypt -- enc
    on_decrypt -- dec
    """
    def __init__(self, *args, master, listbox=None, **kwargs):
        super().__init__(*args, master=master, **kwargs)
        if not hasattr(self, "_listbox"):
            if listbox is None:
                raise TypeError("listbox is required")
            self._listbox = listbox  # don't pack: EIBTI

        self._bencrypt = ttk.Button(
            self,
            text="Encrypt",
            command=self.on_encrypt,
        )
        self._bdecrypt = ttk.Button(
            self,
            text="Decrypt",
            command=self.on_decrypt,
        )
        
        self._bencrypt.pack()
        self._bdecrypt.pack()
    
    def on_encrypt(self):
        """Encrypt everything in the listbox."""
    
    def on_decrypt(self):
        """Decrypt everything in the listbox."""


## Setting up EncDecFrame + ARCFrame

In [130]:
class ControlFrame(ttk.Frame):
    """Full control of listbox. Set up as required.
    
    - Resize the listbox to correct size.
    - Place ARC buttons below the listbox in this way:
        
        [Add, Remove, Remove All]

    - Place the password entry (controlled by EncDecFrame)
    - Place the Enc and Dec buttons in this way:
    
        [Encrypt, Decrypt]
    """
    def __init__(self, *args, master, **kwargs):
        self._listbox = ListBox(*args, master=master, **kwargs)
        self._listbox.pack(
            padx=20,
            pady=20,
            ipadx=200,
            ipady=200,
            expand=True,
            fill=tk.X,
            anchor=tk.N,
        )

        ARCFrame(*args, master=master, listbox=self._listbox, **kwargs).pack(pady=15)
        EncDecFrame(*args, master=master, listbox=self._listbox, **kwargs).pack()
        
        super().__init__(*args, master=master, **kwargs)

In [131]:
root = tk.Tk()
cf = ControlFrame(master=root)

In [132]:
cf.pack()
cf.mainloop()